In [1]:
from collections import Counter
import matplotlib.pyplot as plt
from matplotlib import rc
import numpy as np
import os
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics 
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import matplotlib.transforms as transforms

In [2]:
repo_path = '.'
slurm_path = 'slurm_log'
dict_name = 'summary_jobs.csv'

In [3]:
df = pd.read_csv(os.path.join(repo_path, slurm_path, dict_name))
state = df['State']
print(Counter(state))

Counter({'COMPLETED': 13, 'CANCELLED': 3, '0': 1})


In [4]:
# df_completed = df[(df.State == 'COMPLETED')]
# df_completed.describe()

df_completed = df

In [5]:
print(df_completed.columns)
display(df_completed)

Index(['batch', 'n_trainablebackbone', 'split', 'epochs', 'lr', 'seed',
       'weight', 'test_iou', 'train_iou', 'train_loss', 'Job ID', 'State',
       'Nodes', 'Cores per node', 'CPU Utilized', 'CPU Efficiency',
       'Job Wall-clock time', 'Memory Utilized', 'Memory Efficiency',
       'Energy Consumed', 'Average node power draw', 'autocast', 'backbone',
       'grayscale', 'normalize', 'optimizer'],
      dtype='object')


,batch,n_trainablebackbone,split,epochs,lr,seed,weight,test_iou,train_iou,train_loss,...,Job Wall-clock time,Memory Utilized,Memory Efficiency,Energy Consumed,Average node power draw,autocast,backbone,grayscale,normalize,optimizer
0,1,5,0.8,10,0.0001,42,Non,0.442860,0.476380,1.025040,...,00:40:10,4.30,0.0,1525644,633.047303,0,0,0,0,0
1,1,5,0.8,10,0.0001,42,COCO_V1,0.482060,0.473390,1.009740,...,00:39:20,4.24,0.0,1494536,633.277966,0,0,0,0,0
2,2,3,0.8,20,0.0001,42,COCO_V1,0.491570,0.533860,0.859540,...,01:14:26,4.99,0.0,2859904,640.372593,0,0,0,0,0
3,2,3,0.8,20,0.0001,42,DEFAULT,0.621820,0.691910,0.646540,...,01:17:27,5.01,0.0,3059269,658.332042,0,0,0,0,0
4,2,3,0.8,20,0.0001,42,DEFAULT,0.619460,0.703610,0.625560,...,01:17:13,5.11,0.0,3054722,659.339953,0,0,0,Tru,0
5,2,3,0.9,20,0.0010,42,DEFAULT,0.556200,0.575960,0.885950,...,01:15:34,5.04,0.0,2869602,632.907367,True,0,0,Tru,ada
6,2,3,0.9,12,0.0001,42,DEFAULT,0.565990,0.563750,1.017990,...,00:41:15,20.11,0.0,1736833,701.750707,True,resnet50,0,Tru,ada
7,2,3,0.9,15,0.0001,42,DEFAULT,0.648530,0.647900,0.827440,...,01:11:13,19.72,0.0,2891692,676.735783,True,resnet50,Tru,Tru,ada
8,2,3,0.9,15,0.0001,42,DEFAULT,0.633040,0.649520,0.814930,...,01:11:18,19.83,0.0,2886299,674.684198,True,resnet50,Tru,Tru,ada
9,2,3,0.9,20,0.0001,1,DEFAULT,0.634150,0.696220,0.693740,...,01:34:01,19.87,0.0,3827207,678.462507,True,resnet50,Tru,Tru,ada


In [6]:
features_table = ['test_iou', 'train_iou', 'train_loss',
            'batch',  'weight', 'epochs', 'n_trainablebackbone', 
            'normalize', 'optimizer', 'backbone', 'autocast', 
            'grayscale', 'lr', 'seed', 'split',
            'Nodes', 'Cores per node', 'CPU Efficiency',
            'Memory Utilized','Energy Consumed', 'Average node power draw']

In [7]:
df_completed.drop(df_completed.tail(1).index,inplace=True) # drop last n rows
display(df_completed[features_table])

,test_iou,train_iou,train_loss,batch,weight,epochs,n_trainablebackbone,normalize,optimizer,backbone,...,grayscale,lr,seed,split,Nodes,Cores per node,CPU Efficiency,Memory Utilized,Energy Consumed,Average node power draw
0,0.442860,0.476380,1.025040,1,Non,10,5,0,0,0,...,0,0.0001,42,0.8,1,152,0.68,4.30,1525644,633.047303
1,0.482060,0.473390,1.009740,1,COCO_V1,10,5,0,0,0,...,0,0.0001,42,0.8,1,152,0.68,4.24,1494536,633.277966
2,0.491570,0.533860,0.859540,2,COCO_V1,20,3,0,0,0,...,0,0.0001,42,0.8,1,152,0.68,4.99,2859904,640.372593
3,0.621820,0.691910,0.646540,2,DEFAULT,20,3,0,0,0,...,0,0.0001,42,0.8,1,152,0.68,5.01,3059269,658.332042
4,0.619460,0.703610,0.625560,2,DEFAULT,20,3,Tru,0,0,...,0,0.0001,42,0.8,1,152,0.68,5.11,3054722,659.339953
5,0.556200,0.575960,0.885950,2,DEFAULT,20,3,Tru,ada,0,...,0,0.0010,42,0.9,1,152,0.68,5.04,2869602,632.907367
6,0.565990,0.563750,1.017990,2,DEFAULT,12,3,Tru,ada,resnet50,...,0,0.0001,42,0.9,1,152,1.29,20.11,1736833,701.750707
7,0.648530,0.647900,0.827440,2,DEFAULT,15,3,Tru,ada,resnet50,...,Tru,0.0001,42,0.9,1,152,1.30,19.72,2891692,676.735783
8,0.633040,0.649520,0.814930,2,DEFAULT,15,3,Tru,ada,resnet50,...,Tru,0.0001,42,0.9,1,152,1.30,19.83,2886299,674.684198
9,0.634150,0.696220,0.693740,2,DEFAULT,20,3,Tru,ada,resnet50,...,Tru,0.0001,1,0.9,1,152,1.30,19.87,3827207,678.462507


In [8]:
features = ['test_iou', 'train_iou', 'train_loss', 'optimizer',
            'batch',  'weight', 'epochs', 'n_trainablebackbone', 
            'normalize', 'autocast', 'backbone', 
            'grayscale', 'lr', 'seed', 'split', 'CPU Efficiency',
            'Memory Utilized', 'Average node power draw', 'Energy Consumed']

In [9]:
df_plot = df_completed

In [10]:
# Replace strings by values and old values by the real values
df_plot.loc[df_plot['normalize'] == 'Tru', 'normalize'] = 1
df_plot.loc[df_plot['autocast'] == 'True', 'autocast'] = 1
df_plot.loc[df_plot['grayscale'] == 'Tru', 'grayscale'] = 1
df_plot.loc[df_plot['backbone'] == 'resnet50', 'backbone'] = 0
df_plot.loc[df_plot['backbone'] == 'resnet18', 'backbone'] = 1
df_plot.loc[df_plot['weight'] == 'Non', 'weight'] = 0
df_plot.loc[df_plot['weight'] == 'COCO_V1', 'weight'] = 1
df_plot.loc[df_plot['weight'] == 'DEFAULT', 'weight'] = 2
df_plot.loc[df_plot['optimizer'] == 'ada', 'optimizer'] = 0
df_plot.loc[df_plot['optimizer'] == 'sgd', 'optimizer'] = 1

In [11]:
display(df_plot[features])

,test_iou,train_iou,train_loss,optimizer,batch,weight,epochs,n_trainablebackbone,normalize,autocast,backbone,grayscale,lr,seed,split,CPU Efficiency,Memory Utilized,Average node power draw,Energy Consumed
0,0.442860,0.476380,1.025040,0,1,0,10,5,0,0,0,0,0.0001,42,0.8,0.68,4.30,633.047303,1525644
1,0.482060,0.473390,1.009740,0,1,1,10,5,0,0,0,0,0.0001,42,0.8,0.68,4.24,633.277966,1494536
2,0.491570,0.533860,0.859540,0,2,1,20,3,0,0,0,0,0.0001,42,0.8,0.68,4.99,640.372593,2859904
3,0.621820,0.691910,0.646540,0,2,2,20,3,0,0,0,0,0.0001,42,0.8,0.68,5.01,658.332042,3059269
4,0.619460,0.703610,0.625560,0,2,2,20,3,1,0,0,0,0.0001,42,0.8,0.68,5.11,659.339953,3054722
5,0.556200,0.575960,0.885950,0,2,2,20,3,1,1,0,0,0.0010,42,0.9,0.68,5.04,632.907367,2869602
6,0.565990,0.563750,1.017990,0,2,2,12,3,1,1,0,0,0.0001,42,0.9,1.29,20.11,701.750707,1736833
7,0.648530,0.647900,0.827440,0,2,2,15,3,1,1,0,1,0.0001,42,0.9,1.30,19.72,676.735783,2891692
8,0.633040,0.649520,0.814930,0,2,2,15,3,1,1,0,1,0.0001,42,0.9,1.30,19.83,674.684198,2886299
9,0.634150,0.696220,0.693740,0,2,2,20,3,1,1,0,1,0.0001,1,0.9,1.30,19.87,678.462507,3827207


In [12]:
plt.rcParams["font.family"] = "cmr10"
fig, ax = plt.subplots(1, 1, figsize=(12, 4))
plt.rc('axes', axisbelow=True)
fontsize = 17
legendsize = 12
ticksize = 14
fig.subplots_adjust(right=0.7)

par1 = ax.twinx()
par2 = ax.twinx()

# move the spine of the second axes outwards
par2.spines["right"].set_position(("axes", 1.2))


ax.scatter(np.arange(len(df_plot['test_iou'])), df_plot['test_iou'], label='Test', color='orangered', zorder=10, marker='*', s=70)
ax.scatter(np.arange(len(df_plot['train_iou'])), df_plot['train_iou'], label='Training', color='lightseagreen', zorder=10, marker='s', s=60)
ax.plot(df_plot['test_iou'], color='black', zorder=5, linewidth=0.5)
ax.plot(df_plot['train_iou'], color='black', zorder=5, linewidth=0.5)

p2, = par1.plot(df_plot['epochs'], color='blue')
p3, = par2.plot(df_plot['Energy Consumed']*1e-6, color='green')

ax.set_xlim([0, len(df_plot['test_iou'])])
ax.tick_params(axis='x', labelsize=ticksize)
ax.tick_params(axis='y', labelsize=ticksize)
par1.tick_params(axis='x', labelsize=ticksize)
par1.tick_params(axis='y', labelsize=ticksize)
par2.tick_params(axis='x', labelsize=ticksize)
par2.tick_params(axis='y', labelsize=ticksize)

ax.set_xlabel('Completed and meaningful runs', fontsize=fontsize)
ax.set_ylabel("Intersection-over-Union", fontsize=fontsize)
par1.set_ylabel("Epochs", fontsize=fontsize)
par2.set_ylabel("Energy Consumed [MWhours]", fontsize=fontsize)

leg =ax.legend(facecolor='white', framealpha=1, fontsize=legendsize)
leg.remove()
par2.add_artist(leg)

plt.grid()
ax.axvspan(0, 6, alpha=0.3, color='gray', zorder=1)
ax.axvspan(12, 15, alpha=0.3, color='gray', zorder=1)

lines = [p2, p3]
ax.legend(lines, [l.get_label() for l in lines])

for ax in [par1, par2]:
    ax.set_frame_on(True)
    ax.patch.set_visible(False)

    plt.setp(ax.spines.values(), visible=False)
    ax.spines["right"].set_visible(True)

# ax.yaxis.label.set_color(p1.get_color())
par1.yaxis.label.set_color(p2.get_color())
par2.yaxis.label.set_color(p3.get_color())

par1.spines["right"].set_edgecolor(p2.get_color())
par2.spines["right"].set_edgecolor(p3.get_color())

# ax.tick_params(axis='y', colors=p1.get_color())
par1.tick_params(axis='y', colors=p2.get_color())
par2.tick_params(axis='y', colors=p3.get_color())

plt.savefig(os.path.join('..', 'slides', 'iou_epochs_energy.pdf'), bbox_inches='tight')
plt.clf()

/scratch/ipykernel_509623/821945821.py:46: UserWarning: The label '_child0' of <matplotlib.lines.Line2D object at 0x1465a6150910> starts with '_'. It is thus excluded from the legend.
  ax.legend(lines, [l.get_label() for l in lines])
/scratch/ipykernel_509623/821945821.py:46: UserWarning: The label '_child0' of <matplotlib.lines.Line2D object at 0x1465a6150df0> starts with '_'. It is thus excluded from the legend.
  ax.legend(lines, [l.get_label() for l in lines])


<Figure size 1200x400 with 0 Axes>

In [13]:
def legend(ax, x0=1,y0=1, direction = "v", padpoints = 3,**kwargs):
    otrans = ax.figure.transFigure
    t = ax.legend(bbox_to_anchor=(x0,y0), loc=1, bbox_transform=otrans,**kwargs)
    plt.tight_layout(pad=0)
    ax.figure.canvas.draw()
    plt.tight_layout(pad=0)
    ppar = [0,-padpoints/72.] if direction == "v" else [-padpoints/72.,0] 
    trans2=transforms.ScaledTranslation(ppar[0],ppar[1],fig.dpi_scale_trans)+\
             ax.figure.transFigure.inverted() 
    tbox = t.get_window_extent().transformed(trans2 )
    bbox = ax.get_position()
    if direction=="v":
        ax.set_position([bbox.x0, bbox.y0,bbox.width, tbox.y0-bbox.y0]) 
    else:
        ax.set_position([bbox.x0 + 1, bbox.y0,tbox.x0-bbox.x0, bbox.height]) 

In [14]:
plt.rcParams["font.family"] = "cmr10"
fig, ax = plt.subplots(1, 1, figsize=(10, 3))
plt.rc('axes', axisbelow=True)
fig.subplots_adjust(right=0.85)
fontsize = 15
legendsize = 12
ticksize = 12

plt.scatter(np.arange(len(df_plot['test_iou'])), df_plot['test_iou'], label='Test', color='orangered', zorder=10, marker='*', s=70)
plt.plot(df_plot['test_iou'], color='black', zorder=5, linewidth=0.5)

plt.grid()
ax.axvspan(0, 6, alpha=0.3, color='gray', zorder=1)
ax.axvspan(12, 15, alpha=0.3, color='gray', zorder=1)
plt.xlim([0, len(df_plot['test_iou'])])
plt.xlabel('Completed and meaningful runs', fontsize=fontsize)
plt.ylabel('Intersection-over-Union', fontsize=fontsize)
ax.tick_params(axis='x', labelsize=ticksize)
ax.tick_params(axis='y', labelsize=ticksize)

ax2 = ax.twinx()
ax2.plot(df_plot['optimizer'].astype('float'), label='optimizer (adam or sgd)')
ax2.plot(df_plot['weight'].astype('float'), label='weight (none, coco, default)')
ax2.plot(df_plot['grayscale'].astype('float'), label='grayscale (rgb yes or no)')
ax2.plot(df_plot['backbone'].astype('float'), label='backbone (resnet50 or resnet18)')
ax2.plot(df_plot['autocast'].astype('float'), label='autocast (yes or no)')
ax2.plot(df_plot['n_trainablebackbone'].astype('float'), label='n trainable')

ax2.set_ylabel('Hyperparameters', fontsize=fontsize)
ax2.tick_params(axis='y', labelsize=ticksize)
ax2.set_xlim([0, len(df_plot['test_iou'])])

# leg =ax2.legend(facecolor='white', framealpha=1, fontsize=legendsize, loc='upper left', bbox_to_anchor=(1, 0.5))
legend(ax2, y0=1, direction="h", borderaxespad=0.4, fontsize=legendsize)
# leg.remove()
# ax2.add_artist(leg)

plt.savefig(os.path.join('..', 'slides', 'iou_hyperparams.pdf'), bbox_inches='tight')
plt.clf()

<Figure size 1000x300 with 0 Axes>

In [15]:
plt.rcParams["font.family"] = "cmr10"
fig, ax = plt.subplots(1, 1, figsize=(10, 4))
plt.rc('axes', axisbelow=True)
fig.subplots_adjust(right=0.85)
fontsize = 15
legendsize = 12
ticksize = 14

plt.plot(np.cumsum(df_plot['Energy Consumed']*1e-6))
ax.tick_params(axis='x', labelsize=ticksize)
ax.tick_params(axis='y', labelsize=ticksize)
plt.xlabel('Completed and meaningful runs', fontsize=fontsize)
plt.ylabel("Energy Consumed [MWhours]", fontsize=fontsize)
plt.xlim([0, len(df_plot['test_iou'])])
plt.grid()
ax.axvspan(0, 6, alpha=0.3, color='gray', zorder=1)
ax.axvspan(12, 15, alpha=0.3, color='gray', zorder=1)

plt.savefig(os.path.join('..', 'slides', 'cumulated_energy.pdf'), bbox_inches='tight')
plt.clf()

<Figure size 1000x400 with 0 Axes>

In [16]:
# Feature importance with Pearson's correlation matrix
plt.figure(figsize=(12,12))
cor = df_completed[features].corr()  # method = kendall, pearson, spearman
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.savefig(os.path.join('..', 'slides', 'cor_matrix.pdf'), bbox_inches='tight')
plt.clf()

/hkfs/work/workspace/scratch/ih5525-E4/AI-HERO-2-Energy/energy_env/lib64/python3.9/site-packages/seaborn/utils.py:80: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from current font.
  fig.canvas.draw()
/scratch/ipykernel_509623/2902151204.py:5: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from current font.
  plt.savefig(os.path.join('..', 'slides', 'cor_matrix.pdf'), bbox_inches='tight')
/scratch/ipykernel_509623/2902151204.py:5: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from current font.
  plt.savefig(os.path.join('..', 'slides', 'cor_matrix.pdf'), bbox_inches='tight')


<Figure size 1200x1200 with 0 Axes>

In [308]:
# Feature importance with random forest, all data
# https://www.geeksforgeeks.org/hyperparameters-of-random-forest-classifier/
clf = RandomForestClassifier(n_estimators = 20, max_depth=20, min_samples_split=4, random_state=1)
x = df_completed[features]
y = df_completed[['test_iou']]
clf.fit(x, y)
feature_imp = pd.Series(clf.feature_importances_, index = features).sort_values(ascending = False)
feature_imp

/scratch/ipykernel_160355/3151318444.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(x, y)


ValueError: Unknown label type: 'continuous'

In [ ]:
# Feature importance with random forest, train and test data

# Create training and test split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state=1)

# Feature scaling
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# Training / Test Dataframe
X_train_std = pd.DataFrame(X_train_std, columns=features)
X_test_std = pd.DataFrame(X_test_std, columns=features)

forest = RandomForestClassifier(n_estimators=20, max_depth=5, min_samples_split=2, random_state=1)
forest.fit(X_train_std, y_train.values.ravel())

feature_imp = pd.Series(forest.feature_importances_, index = features).sort_values(ascending = False)
feature_imp

n_trainablebackbone        0.203333
test_iou                   0.186667
train_iou                  0.176667
train_loss                 0.136667
Energy Consumed            0.130000
Memory Utilized            0.090000
Average node power draw    0.076667
lr                         0.000000
epochs                     0.000000
batch                      0.000000
seed                       0.000000
Cores per node             0.000000
Nodes                      0.000000
split                      0.000000
CPU Efficiency             0.000000
dtype: float64